<a href="https://colab.research.google.com/github/HassanSherwani/Product_Purchase_Frequency/blob/master/Product_Freq.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Problem Statement:

How often a certain product has been sold in the past days.

#1)- Importing key modules

In [0]:
# Let's be rebels and ignore warnings for now
import warnings
warnings.filterwarnings('ignore')
warnings.filterwarnings("ignore",category=DeprecationWarning)

In [0]:
import pandas as pd
import numpy as np
import time
from sklearn.model_selection import train_test_split
import sys

In [3]:
! pip install turicreate

In [0]:
import turicreate as tc

# 2)-Loading Data

In [0]:
url = 'https://raw.githubusercontent.com/HassanSherwani/Product_Purchase_Frequency/master/20190207_transactions%20.json'

In [0]:
transactions = pd.read_json(url, lines= True)

# 3)-Exploring dataset

In [7]:
transactions.head()

,id,products
0,0,"[185, 30, 77, 188, 78, 125, 45, 155, 241, 229,..."
1,1,"[119, 148, 108, 34, 157, 82, 113, 45, 165]"
2,2,"[173, 103, 229, 240]"
3,3,[91]
4,4,"[175, 192, 54, 172]"


In [8]:
transactions.shape

(2500, 2)

In [9]:
print(transactions['products'][1415])

[250, 236, 242, 229, 92, 2, 71, 172, 109, 247, 171, 209, 90, 139, 188, 191, 145, 214, 216, 237]


In [10]:
transactions.info() # checking missing values

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2500 entries, 0 to 2499
Data columns (total 2 columns):
id          2500 non-null int64
products    2500 non-null object
dtypes: int64(1), object(1)
memory usage: 138.6+ KB


In [11]:
transactions.describe()

,id
count,2500.00000
mean,1249.50000
std,721.83216
min,0.00000
25%,624.75000
50%,1249.50000
75%,1874.25000
max,2499.00000


# 4)- Adding Features:

### 4.1)-Create a separate dataframe for recommending users

In [0]:
customers=transactions['id']

In [13]:
customers.head()

0    0
1    1
2    2
3    3
4    4
Name: id, dtype: int64

In [0]:
import random
random.shuffle(customers) 

 To get random values of customer ids. This would help us in avoiding sample biaseness in our modeling.

In [0]:
cust_2_rec=customers[:1000]

In [16]:
cust_2_rec.head()

0    1717
1    2016
2    2342
3     404
4    1258
Name: id, dtype: int64

### 4.2)- break down each list of items in the products column into rows and count the number of products bought by a user

In [17]:


pd.melt(transactions.head(2).set_index('id')['products'].apply(pd.Series).reset_index(), 
             id_vars=['id'],
             value_name='products') \
    .dropna().drop(['variable'], axis=1) \
    .groupby(['id', 'products']) \
    .agg({'products': 'count'}) \
    .rename(columns={'products': 'purchase_count'}) \
    .reset_index() \
    .rename(columns={'products': 'productId'})

,id,productId,purchase_count
0,1717,30.0,1
1,1717,45.0,1
2,1717,77.0,1
3,1717,78.0,1
4,1717,89.0,2
5,1717,125.0,1
6,1717,133.0,1
7,1717,155.0,1
8,1717,161.0,1
9,1717,185.0,1


# 5- Creating features for user_id,product_id and purchase count

### 5a)-One for purchase count

In [18]:
s=time.time()

data = pd.melt(transactions.set_index('id')['products'].apply(pd.Series).reset_index(), 
             id_vars=['id'],
             value_name='products') \
    .dropna().drop(['variable'], axis=1) \
    .groupby(['id', 'products']) \
    .agg({'products': 'count'}) \
    .rename(columns={'products': 'purchase_count'}) \
    .reset_index() \
    .rename(columns={'products': 'productId'})
data['productId'] = data['productId'].astype(np.int64)

print("Execution time:", round((time.time()-s)/60,2), "minutes")

Execution time: 0.01 minutes


In [19]:
data.shape

(24811, 3)

In [20]:
data.head()

,id,productId,purchase_count
0,0,46,1
1,1,76,1
2,1,87,1
3,1,147,1
4,1,188,2


### 5-b)-Normalize item

we normalize purchase frequency of each item across users by first creating a user-item matrix 

In [0]:
df_matrix = pd.pivot_table(data, values='purchase_count', index='id', columns='productId')

In [22]:
df_matrix.head()

productId,1,2,3,4,5,6,7,8,9,10,...,241,242,243,244,245,246,247,248,249,250
id,,,,,,,,,,,,,,,,,,,,,
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN


The NaN tells us that the item represented by the column was not purchased in that specific transaction.

In [23]:
df_matrix.shape

(2378, 250)

In [24]:
df_matrix_norm = (df_matrix-df_matrix.min())/(df_matrix.max()-df_matrix.min())

df_matrix_norm.head()

productId,1,2,3,4,5,6,7,8,9,10,...,241,242,243,244,245,246,247,248,249,250
id,,,,,,,,,,,,,,,,,,,,,
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN


In [25]:
print(df_matrix_norm.shape)

(2378, 250)


### 5-c)- create a table for input to the modeling

In [0]:
d = df_matrix_norm.reset_index()
d.index.names = ['scaled_purchase_freq']
data_norm = pd.melt(d, id_vars=['id'], value_name='scaled_purchase_freq').dropna()

In [27]:
data_norm.head()

,id,productId,scaled_purchase_freq
18,19,1,0.0
42,48,1,0.0
73,79,1,0.0
101,107,1,0.0
121,127,1,0.0


In [28]:
data_norm.shape

(22530, 3)

### 5-d) a function for normalizing data

In [0]:
def normalize_data(data):
    df_matrix = pd.pivot_table(data, values='purchase_count', index='id', columns='productId')
    df_matrix_norm = (df_matrix-df_matrix.min())/(df_matrix.max()-df_matrix.min())
    d = df_matrix_norm.reset_index()
    d.index.names = ['scaled_purchase_freq']
    return pd.melt(d, id_vars=['id'], value_name='scaled_purchase_freq').dropna()


we have normalized the items accoreding to their purchase history, from 0–1 (with 1 being the most number of purchase for an item and 0 being 0 purchase count for that item.

# 6)-Split train and test set

we have three dataframes with purchase counts(data), purchase dummy(data_dummy), and scaled purchase counts(data_norm).

In [30]:
train, test = train_test_split(data, test_size = .2)
print(train.shape, test.shape)

(19848, 3) (4963, 3)


In [0]:
#convert dataframe to SFrame
train_data = tc.SFrame(train)
test_data = tc.SFrame(test)

A tabular, column-mutable dataframe object that can scale to big data <br>
https://turi.com/products/create/docs/generated/graphlab.SFrame.html

In [32]:
train_data

id,productId,purchase_count
2392,39,1
2341,22,1
1326,58,1
339,229,1
327,157,1
949,186,1
1235,93,1
1280,62,1
1985,196,1
1958,107,1


In [33]:
test_data

id,productId,purchase_count
1608,62,1
888,210,1
433,37,1
466,187,1
1793,61,1
450,76,1
1224,135,1
2418,17,1
1511,62,1
2125,246,1


### 6.2)- Define a split_data function 

In [0]:
def split_data(data):

    train, test = train_test_split(data, test_size = .2)
    train_data = tc.SFrame(train)
    test_data = tc.SFrame(test)
    return train_data, test_data

### 6.3)-Apply for scaled/normalized purchase table

In [0]:
train_data_norm, test_data_norm = split_data(data_norm)

# 7)-Building Model

### 7.1)- Parameters to define field names for purchase count as target feature

In [0]:
user_id = 'id'
item_id = 'productId'
users_to_recommend=list(cust_2_rec)
target = 'purchase_count'
n_rec = 10 # number of items to recommend
n_display = 30 # 1st 30 rows to display

In [37]:
popularity_model = tc.popularity_recommender.create(train_data, 
                                                    user_id=user_id, 
                                                    item_id=item_id, 
                                                    target=target)

Preparing data set.

Data has 19848 observations with 2348 users and 250 items.

Data prepared in: 0.036782s

19848 observations to process; with 250 unique items.

In [38]:
popularity_recomm = popularity_model.recommend(users=users_to_recommend, k=n_rec)
popularity_recomm.print_rows(n_display)

recommendations finished on 1000/1000 queries. users per second: 98299.4

+------+-----------+--------------------+------+
|  id  | productId |       score        | rank |
+------+-----------+--------------------+------+
| 1717 |    153    | 1.0897435897435896 |  1   |
| 1717 |    201    | 1.0845070422535212 |  2   |
| 1717 |    207    | 1.0813953488372092 |  3   |
| 1717 |     96    | 1.0769230769230769 |  4   |
| 1717 |    131    | 1.0740740740740742 |  5   |
| 1717 |     35    | 1.0704225352112675 |  6   |
| 1717 |     26    | 1.0684931506849316 |  7   |
| 1717 |    152    | 1.065934065934066  |  8   |
| 1717 |    120    | 1.064102564102564  |  9   |
| 1717 |     44    | 1.0632911392405062 |  10  |
| 2016 |    153    | 1.0897435897435896 |  1   |
| 2016 |    201    | 1.0845070422535212 |  2   |
| 2016 |    207    | 1.0813953488372092 |  3   |
| 2016 |     96    | 1.0769230769230769 |  4   |
| 2016 |    131    | 1.0740740740740742 |  5   |
| 2016 |     35    | 1.0704225352112675 |  6   |
| 2016 |     26    | 1.0684931506849316 |  7   |
| 2016 |    152    |

### Note 

Through this model, we predicted the recommendation items using scores by popularity. As you can tell for each model results above, the rows show the first 30 records from 1000 users with 10 recommendations.<br> These 30 records include 3 users and their recommended items, along with score and descending ranks.

## Checking most frequent items(as per assignment) 

In [39]:
 train.groupby(by=item_id)['purchase_count'].mean().sort_values(ascending=False).head(20)

productId
153    1.089744
201    1.084507
207    1.081395
96     1.076923
131    1.074074
35     1.070423
26     1.068493
152    1.065934
120    1.064103
44     1.063291
166    1.062500
49     1.061538
102    1.059524
125    1.059524
170    1.058824
71     1.058824
129    1.056338
150    1.055556
7      1.054945
101    1.054795
Name: purchase_count, dtype: float64

**products 153, 201,207,96 and 131 are the most popular (best-selling) across customers.**

### 7.2)- Applying 'scaled_purchase_freq' as target feature on model

In [0]:
user_id = 'id'
item_id = 'productId'
users_to_recommend=list(cust_2_rec)
target = 'scaled_purchase_freq'
n_rec = 10 # number of items to recommend
n_display = 30 # 1st 30 rows to display

In [41]:
popularity_model_scaled = tc.popularity_recommender.create(train_data_norm, 
                                                    user_id=user_id, 
                                                    item_id=item_id, 
                                                    target=target)

Preparing data set.

Data has 18024 observations with 2337 users and 227 items.

Data prepared in: 0.032764s

18024 observations to process; with 227 unique items.

In [42]:
popularity_recomm_4_scaled = popularity_model_scaled.recommend(users=users_to_recommend, k=n_rec)
popularity_recomm_4_scaled.print_rows(n_display)

recommendations finished on 1000/1000 queries. users per second: 83738.1

+------+-----------+----------------------+------+
|  id  | productId |        score         | rank |
+------+-----------+----------------------+------+
| 1717 |     96    | 0.08641975308641975  |  1   |
| 1717 |    152    | 0.07526881720430108  |  2   |
| 1717 |     35    | 0.07246376811594203  |  3   |
| 1717 |    207    | 0.06976744186046512  |  4   |
| 1717 |    117    |  0.0684931506849315  |  5   |
| 1717 |     44    | 0.06593406593406594  |  6   |
| 1717 |    153    |  0.0641025641025641  |  7   |
| 1717 |     26    |  0.0641025641025641  |  8   |
| 1717 |    201    | 0.06153846153846154  |  9   |
| 1717 |    112    | 0.060240963855421686 |  10  |
| 2016 |     96    | 0.08641975308641975  |  1   |
| 2016 |    152    | 0.07526881720430108  |  2   |
| 2016 |     35    | 0.07246376811594203  |  3   |
| 2016 |    207    | 0.06976744186046512  |  4   |
| 2016 |    117    |  0.0684931506849315  |  5   |
| 2016 |     44    | 0.06593406593406594  |  6   |
| 2016 |    153    |  0.0641025

Great!.... We have all three models worked out. We have purchase frequency of each item customer-vice. <br>
Which one to trust and which one to discard? 

# 8)- Evaluate models

### 8.1)- Validation for Popularity Model on Purchase Counts

In [0]:
models_counts = [popularity_model]

In [0]:
model_names=['Popularity Model on Purchase Counts']

In [45]:
eval_counts = tc.recommender.util.compare_models(test_data, models_counts, model_names)

PROGRESS: Evaluate model Popularity Model on Purchase Counts


recommendations finished on 1000/1912 queries. users per second: 40142.9


Precision and recall summary statistics by cutoff
+--------+----------------------+----------------------+
| cutoff |    mean_precision    |     mean_recall      |
+--------+----------------------+----------------------+
|   1    | 0.010460251046025104 | 0.004881450488145039 |
|   2    | 0.009152719665271978 | 0.007314704124327552 |
|   3    |  0.0097629009762901  | 0.010696851962542345 |
|   4    | 0.010198744769874493 | 0.015730847778441928 |
|   5    | 0.010669456066945608 | 0.01949217971707513  |
|   6    | 0.010373082287308235 | 0.023449641362821298 |
|   7    | 0.01060968320382547  | 0.02923764694162188  |
|   8    | 0.010917887029288711 | 0.03402092880387859  |
|   9    | 0.011215713621571352 | 0.03934320415753462  |
|   10   | 0.012238493723849369 | 0.048495923822806715 |
+--------+----------------------+----------------------+
[10 rows x 3 columns]


Overall RMSE: 0.14789594157530456

Per User RMSE (best)
+------+------+-------+
|  id  | rmse | count |
+------+------+-------+

### 8.2)- Validation for Popularity Model on Scaled Purchase Counts

In [46]:
models_counts = [popularity_model_scaled]
model_names=['Popularity Model on Scaled Purchase Counts']
eval_counts_dummy = tc.recommender.util.compare_models(test_data_norm, models_counts, model_names)

PROGRESS: Evaluate model Popularity Model on Scaled Purchase Counts


recommendations finished on 1000/1846 queries. users per second: 52021


Precision and recall summary statistics by cutoff
+--------+----------------------+----------------------+
| cutoff |    mean_precision    |     mean_recall      |
+--------+----------------------+----------------------+
|   1    | 0.010834236186348873 | 0.004749006861682915 |
|   2    | 0.01137594799566631  | 0.009376773461280501 |
|   3    | 0.011556518598772117 | 0.013616313264200578 |
|   4    | 0.011240520043336973 | 0.016934942991281052 |
|   5    | 0.011700975081256764 | 0.022284992003301873 |
|   6    | 0.011375947995666304 | 0.025554609709539287 |
|   7    | 0.011143785791673124 | 0.029478150957024205 |
|   8    | 0.01124052004333695  | 0.032899963885879395 |
|   9    | 0.010774045985313558 | 0.03564463705308773  |
|   10   | 0.010888407367280604 | 0.03983451994015361  |
+--------+----------------------+----------------------+
[10 rows x 3 columns]


Overall RMSE: 0.1585013876452898

Per User RMSE (best)
+------+------+-------+
|  id  | rmse | count |
+------+------+-------+


 We tend to keep lowest rmse and it signifies least error in prediction. 



**So we will choose that as best model with rmse 0.147 i.e (in 8.1) for our final step.**

# 9) Submission

In [47]:
users_to_recommend = list(cust_2_rec)

final_model = tc.item_similarity_recommender.create(tc.SFrame(data), 
                                            user_id=user_id, 
                                            item_id=item_id, 
                                            target='purchase_count')


Preparing data set.

Data has 24811 observations with 2378 users and 250 items.

Data prepared in: 0.030951s

Training model from provided data.

Gathering per-item and per-user statistics.

...

| 2.532ms                        | 100        |

+--------------------------------+------------+

Setting up lookup tables.

Processing data in one pass using dense lookup tables.

+-------------------------------------+------------------+-----------------+

In [48]:
recom = final_model.recommend(users=users_to_recommend, k=n_rec)
recom.print_rows(n_display)

| Elapsed Time (Constructing Lookups) | Total % Complete | Items Processed |

+-------------------------------------+------------------+-----------------+

| 3.448ms                             | 0                | 0               |

| 22.818ms                            | 100              | 250             |

+-------------------------------------+------------------+-----------------+

Finalizing lookup tables.

Generating candidate set for working with new users.

Finished training in 1.02944s

recommendations finished on 1000/1000 queries. users per second: 30774.9

+------+-----------+----------------------+------+
|  id  | productId |        score         | rank |
+------+-----------+----------------------+------+
| 1717 |    168    | 0.025631868839263917 |  1   |
| 1717 |    248    | 0.025486195087432863 |  2   |
| 1717 |     23    | 0.024235220750172932 |  3   |
| 1717 |     32    | 0.02150735855102539  |  4   |
| 1717 |    214    | 0.021062183380126952 |  5   |
| 1717 |    126    | 0.02020918925603231  |  6   |
| 1717 |     63    | 0.020147351423899333 |  7   |
| 1717 |    139    | 0.019966328144073488 |  8   |
| 1717 |     74    | 0.01874724229176839  |  9   |
| 1717 |    110    | 0.018568030993143716 |  10  |
| 2016 |    177    | 0.030139015780554876 |  1   |
| 2016 |     80    | 0.02526219023598565  |  2   |
| 2016 |    112    | 0.025098118517133925 |  3   |
| 2016 |    122    | 0.02353888087802463  |  4   |
| 2016 |     23    | 0.023420088820987277 |  5   |
| 2016 |     17    | 0.023266335328420002 |  6   |
| 2016 |    150    | 0.02250588

### 9.2)- Checking most frequent items in final model

In [49]:
data.groupby(by=item_id)['purchase_count'].mean().sort_values(ascending=False).head(20)

productId
96     1.080808
207    1.076923
153    1.071429
201    1.069767
26     1.063158
152    1.060345
125    1.058824
112    1.058824
85     1.058824
131    1.057143
35     1.056180
63     1.054545
44     1.054545
129    1.053763
101    1.053763
117    1.052632
150    1.051724
122    1.051724
156    1.051546
120    1.050000
Name: purchase_count, dtype: float64

**RESULT: <BR>
products 96,207 , 153,201 and 26(and so on) are the most frequent selling items across customers.**

# 10)- Bonus Part- Recommending products to customers

In [50]:
# Create a csv output file
df_rec = recom.to_dataframe()
df_rec.head()

,id,productId,score,rank
0,1717,168,0.025632,1
1,1717,248,0.025486,2
2,1717,23,0.024235,3
3,1717,32,0.021507,4
4,1717,214,0.021062,5


In [51]:
print(df_rec.shape)

(10000, 4)


In [0]:
df_rec['recommendedProducts'] = df_rec.groupby([id])[item_id].transform(lambda x: '|'.join(x.astype(str)))
df_output = df_rec[['id', 'recommendedProducts']].drop_duplicates().sort_values('id').set_index('id')

In [53]:
recomendation = final_model.recommend(users=users_to_recommend, k=n_rec)

recommendations finished on 1000/1000 queries. users per second: 34078.5

In [0]:
df_rec = recomendation.to_dataframe()


In [0]:
df_rec['recommendedProducts'] = df_rec.groupby([user_id])[item_id] \
        .transform(lambda x: '|'.join(x.astype(str)))

In [0]:

df_output = df_rec[['id', 'recommendedProducts']].drop_duplicates() \
        .sort_values('id').set_index('id')

In [57]:
df_output.head()

,recommendedProducts
id,
0,184|219|150|181|82|24|58|115|101|147
1,29|137|51|94|119|235|83|229|28|221
2,54|100|195|45|23|200|211|152|36|248
3,250|36|121|234|50|65|174|153|235|77
6,228|184|227|175|137|87|69|164|201|243
